In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer

import numpy  as np
import tflearn
import tensorflow
import random
import json
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [3]:
with open("C:/Users/kales/New_dataset/intents.json") as file:
    data=json.load(file)
print(data)
print(data["intents"])

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [4]:
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
    
print(words)

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'dont', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'entry', 'I', 'want', 'to', 'log', 'blood', 'pr

In [5]:
words = [ps.stem(w.lower()) for w in words if w != "?" ]
words = sorted(list(set(words)))

labels = sorted(labels)

training=[]
output=[]
out_empty= [0 for _ in range(len(labels))]

In [6]:
for x, doc in enumerate(docs_x):
    bag=[]
    wrds=[ps.stem(w1) for w1 in doc]
    for w1 in words:
        if w1 in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1
    
    training.append(bag)
    output.append(output_row)
    
training=np.array(training)
output=np.array(output)


In [7]:
tensorflow.reset_default_graph()  
net = tflearn.input_data(shape=[None,len(training[0])])  
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation="softmax")
net = tflearn.regression(net)   

model = tflearn.DNN(net)

model.fit(training,output,n_epoch=1000,batch_size=8,show_metric=True)
model.save("modelchabot")

Training Step: 5999  | total loss: 0.00861 | time: 0.025s
| Adam | epoch: 1000 | loss: 0.00861 - acc: 1.0000 -- iter: 40/47
Training Step: 6000  | total loss: 0.00868 | time: 0.030s
| Adam | epoch: 1000 | loss: 0.00868 - acc: 1.0000 -- iter: 47/47
--
INFO:tensorflow:C:\Users\kales\modelchabot is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
def bag_of_words(s,words):
    bag=[0 for _ in range (len(words))]
    s_words= nltk.word_tokenize(s)
    s_words=[ps.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w==se:
                bag[i] =1
    return np.array(bag)

def chat():
    print("starting talking with the bot (type quit to")
    while True:
        inp= input('You: ')
        if inp.lower()=='quit':
            break
        results=model.predict([bag_of_words(inp, words)])
        result_index=np.argmax(results)
        tag=labels[result_index]
        #print(tag)
        
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses=tg['responses']
        print(random.choice(responses))
chat()



starting talking with the bot (type quit to
You: Hi there
Hello, thanks for asking
You: hello
Hi there, how can I help?
You: How to check Adverse drug reaction?
Navigating to Adverse drug reaction module
You: hi
Hi there, how can I help?
You: Open blood pressure module
Navigating to Blood Pressure module
You: search_blood_pressure_by_patient_id
Hello, thanks for asking
You: Find me a pharmacy
Please provide pharmacy name
You: quit
